In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import sys
print('python', sys.version)

import os

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
import scipy.stats

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
numpy 1.19.1
pandas 1.1.1
matplotlib 3.3.1


## mapping human gene symbol to entrez

In [3]:
# mapping human gene symbol to entrez
dat=pd.read_table('data_ori/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int).astype(str)

e2s=dat.set_index('entrez')
e2s_dic=e2s['sym'].to_dict() # e2s_dic={'1': 'A1BG', '503538': 'A1BG-AS1': ,...}

## GI prediction (with shRNA)

### read expression file

In [4]:
exp_ori=pd.read_table('data_ori/CCLE_expression.csv',index_col=0, sep=',') # CCLE_expression, 19144 genes, 1305 cells

print('# of nan: ', exp_ori.isnull().sum().sum()) # not containing nan value

exp_ori.columns=[gene.split(' ')[1] for gene in exp_ori.columns]
exp_ori.columns=[col[1:-1] for col in exp_ori.columns]
exp_ori.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in exp_ori.columns]
exp_ori=exp_ori.loc[:,exp_ori.columns!='NA_symbol']

exp_arr=np.array(exp_ori)
print('# of ori expresions: ', exp_arr.flatten().shape)

exp_nonzero=exp_arr[exp_arr!=0]
print('# of non-zero expresions: ', exp_nonzero.shape)

exp_ori.index.name='DepMap_ID'
exp_ori.head(3)

# of nan:  0
# of ori expresions:  (24975090,)
# of non-zero expresions:  (20781966,)


,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,SCYGR3,SCYGR7,OR8S1,SCYGR6,OR4F29,EEF1AKMT4,TBCE,OOSP4A,ARHGAP11B,OOSP3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-001097,0.000000,0.0,4.667324,1.761285,3.554589,4.358959,0.641546,2.201634,1.941106,2.150560,...,0.0,0.0,0.0,0.0,0.0,2.980025,5.132988,0.0,0.443607,0.000000
ACH-001636,0.000000,0.0,6.198887,2.032101,3.755956,2.601697,0.000000,0.000000,3.415488,3.620586,...,0.0,0.0,0.0,0.0,0.0,4.322649,4.100978,0.0,2.589763,0.298658
ACH-001804,4.934988,0.0,6.635464,1.929791,2.503349,0.000000,4.883621,6.772546,3.889474,4.139961,...,0.0,0.0,0.0,0.0,0.0,2.283922,4.288359,0.0,0.475085,0.000000


### read mutation profile

In [5]:
mut_cnt=pd.read_table('data_preproc/mut_varL.txt',index_col=0, sep='\t')

mut_cnt.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in mut_cnt.columns]
mut_cnt=mut_cnt.loc[:,mut_cnt.columns!='NA_symbol']

mut_cnt.head(3)

,A1BG,A2M,NAT1,NAT2,SERPINA3,AADAC,AAMP,AANAT,AARS1,ABAT,...,PTGES3L,DNAH17-AS1,FMC1-LUC7L2,PYURF,OVCH1-AS1,GOLGA8R,NPIPA8,ZNF891,SPECC1L-ADORA2A,MEI4
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### read depletion profile

In [10]:
dep_shr=pd.read_table('data_preproc/dep_shRNA.txt',index_col=0, sep='\t')

dep_shr.columns=[col[1:-1] for col in dep_shr.columns]
dep_shr.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in dep_shr.columns]
dep_shr=dep_shr.loc[:,dep_shr.columns!='NA_symbol']

dep_shr.head(3)

,CDH2,AKT3,MED6,NR2E3,SIGLEC14,KCNE3,ZBTB33,CDH3,CDC42P3,HDAC6,...,NR1H4,CCS,NR1D2,CDC42,RCE1,HNRNPDL,DMTF1,PPP4R1,KCNE2,CASP8AP2
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-001270,-0.194962,-0.256108,-0.174220,-0.140052,0.077742,-0.235161,-0.012051,0.135390,0.197630,-0.042870,...,0.077694,0.224705,0.178494,-0.273734,-0.201644,-0.363670,0.184260,-0.115616,0.088853,-0.843295
ACH-001000,-0.028171,0.100751,-0.456124,-0.174618,-0.111691,-0.057127,-0.036372,0.019846,0.053259,0.022057,...,-0.120004,0.249664,-0.057862,-0.274488,0.074889,0.152158,0.036011,0.117300,-0.110628,-0.307031
ACH-001001,0.063047,-0.008077,-0.214376,-0.153619,-0.195624,0.143540,-0.003294,0.137083,-0.184220,0.000256,...,0.265685,-0.003567,0.064435,-0.065205,0.006735,-0.033385,0.197651,-0.016372,0.057286,-0.413669


### get shared cells between mut_count and dep_crisper

In [11]:
shared_index=set(mut_cnt.index) & set(dep_shr.index) & set(exp_ori.index)
shared_columns=set(dep_shr.columns) & set(exp_ori.columns)
mut_sh=mut_cnt.loc[shared_index]
dep_sh=dep_shr.loc[shared_index, shared_columns]
exp_sh=exp_ori.loc[shared_index, shared_columns]

mut_sh.head(3)
mut_sh.shape
dep_sh.head(3)
dep_sh.shape
exp_sh.head(3)
exp_sh.shape

,A1BG,A2M,NAT1,NAT2,SERPINA3,AADAC,AAMP,AANAT,AARS1,ABAT,...,PTGES3L,DNAH17-AS1,FMC1-LUC7L2,PYURF,OVCH1-AS1,GOLGA8R,NPIPA8,ZNF891,SPECC1L-ADORA2A,MEI4
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000846,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(649, 18725)

,USP10,EPB41L4A,VWF,ZNF180,PI4K2A,MPG,GNRHR,SERPINE1,JUNB,PANK1,...,TXK,SMPD1,CYSLTR1,DLAT,TBCE,CPB2,SEMA5A,HECW2,CERK,ABCB9
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000330,-0.022853,-0.131727,-0.023480,0.012869,0.162434,0.066801,-0.285436,0.014675,-0.165101,0.052157,...,0.016496,-0.011611,-0.031725,-0.076238,-0.265114,0.043126,-0.046625,-0.101576,0.206704,-0.119655
ACH-000055,0.058069,-0.257214,-0.367417,-0.136605,-0.105753,0.126095,0.287164,-0.290602,-0.011513,-0.195763,...,0.178259,0.108189,-0.016311,0.130278,-0.704258,0.403621,0.072294,-0.787489,0.139844,-0.325665
ACH-000846,-0.128131,0.040671,0.166831,0.125949,-0.040917,0.010232,-0.271800,-0.071253,-0.060397,0.147677,...,0.050562,-0.045902,0.048681,-0.146970,-0.766374,0.088043,0.093462,-0.097076,0.019499,-0.144523


(649, 5688)

,USP10,EPB41L4A,VWF,ZNF180,PI4K2A,MPG,GNRHR,SERPINE1,JUNB,PANK1,...,TXK,SMPD1,CYSLTR1,DLAT,TBCE,CPB2,SEMA5A,HECW2,CERK,ABCB9
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000330,5.707635,0.731183,1.748461,3.007196,2.134221,5.227664,0.028569,0.443607,6.509379,2.341986,...,0.056584,2.280956,0.042644,4.528571,4.478972,0.070389,0.014355,0.344828,4.165912,4.060047
ACH-000055,4.925999,1.286881,0.124328,2.277985,3.424922,4.311067,0.150560,0.443607,4.793376,2.788686,...,0.056584,1.304511,0.124328,5.082788,6.524816,0.000000,0.056584,2.618239,4.462052,1.773996
ACH-000846,6.668460,2.000000,1.669027,1.899176,4.412782,5.307064,0.111031,2.269033,6.439291,2.555816,...,0.310340,1.691534,0.000000,5.137094,4.783980,0.097611,0.014355,0.275007,5.333782,3.411426


(649, 5688)

### GI score by ttest

In [12]:
def perform_ttest(mut_sel, dep_sel, exp_sel):
    
    meta_info='shRNA_M{}_I{}'.format(str(MIN_MUT_PER).replace('.',''), MIN_MUT_NUM)
    
    # open file for SL score
    SL_WO_Exp=open('result_SL/{}_WO_Exp.txt'.format(meta_info),'w')
    SL_WI_Exp=open('result_SL/{}_WI_Exp.txt'.format(meta_info),'w')
    _=SL_WO_Exp.write('\t'.join(['mut_gene','dep_gene','t_val','p_val'])+'\n')
    _=SL_WI_Exp.write('\t'.join(['mut_gene','dep_gene','t_val','p_val'])+'\n')

    # select genes mutated frequently
    mut_num=np.sum(mut_sel!=0, axis=0)
    mut_per=np.sum(mut_sel!=0, axis=0)/len(mut_sel)
    mut_sel=mut_sel.loc[:,(mut_num>=MIN_MUT_NUM)&(mut_per>=MIN_MUT_PER)]
    
    mut_g_cnt=0
    for mut_g in mut_sel.columns:
        mut_g_cnt+=1
        print('processing : {}/{}'.format(mut_g_cnt, len(mut_sel.columns)))
        mut_sr=mut_sel[mut_g]
        
        for dep_g in list(dep_sel):
            dep_sr=dep_sel[dep_g]
            exp_sr=exp_sel[dep_g]
            
            # merge mutation, depletion, expression
            mer_df=pd.concat([mut_sr,dep_sr,exp_sr], axis=1, join='inner')
            mer_df.columns=['mut','dep','exp']

            # ttest without considering expression data
            con_dep_score=mer_df.loc[mer_df['mut']==0, 'dep']
            mut_dep_score=mer_df.loc[mer_df['mut']!=0, 'dep']
                        
            t_val, p_val = scipy.stats.ttest_ind(con_dep_score, mut_dep_score, equal_var=False)
            _=SL_WO_Exp.write('{}\t{}\t{:.5f}\t{:.5e}\n'.format(mut_g, dep_g, t_val, p_val))
            
            # ttest with considering expression data
            # obtain instance with nonzero expression
            mer_df=mer_df.loc[mer_df['exp']!=0]
            con_dep_score=mer_df.loc[mer_df['mut']==0, 'dep']
            mut_dep_score=mer_df.loc[mer_df['mut']!=0, 'dep']
            
            if (len(mut_dep_score) < MIN_MUT_NUM) or (len(mut_dep_score)/len(mer_df) < MIN_MUT_PER):
                continue
                
            t_val, p_val = scipy.stats.ttest_ind(con_dep_score, mut_dep_score, equal_var=False)
            _=SL_WI_Exp.write('{}\t{}\t{:.5f}\t{:.5e}\n'.format(mut_g, dep_g, t_val, p_val))
        
    SL_WO_Exp.close()
    SL_WI_Exp.close()

In [13]:
MIN_MUT_PER=0.03 ## minimum mutation occurance percentage
MIN_MUT_NUM=10

perform_ttest(mut_sh, dep_sh, exp_sh) # t_val: positive --> SL signature    

processing : 1/5981
processing : 2/5981
processing : 3/5981
processing : 4/5981
processing : 5/5981
processing : 6/5981
processing : 7/5981
processing : 8/5981
processing : 9/5981
processing : 10/5981
processing : 11/5981
processing : 12/5981
processing : 13/5981
processing : 14/5981
processing : 15/5981
processing : 16/5981
processing : 17/5981
processing : 18/5981
processing : 19/5981
processing : 20/5981
processing : 21/5981
processing : 22/5981
processing : 23/5981
processing : 24/5981
processing : 25/5981
processing : 26/5981
processing : 27/5981
processing : 28/5981
processing : 29/5981
processing : 30/5981
processing : 31/5981
processing : 32/5981
processing : 33/5981
processing : 34/5981
processing : 35/5981
processing : 36/5981
processing : 37/5981
processing : 38/5981
processing : 39/5981
processing : 40/5981
processing : 41/5981
processing : 42/5981
processing : 43/5981
processing : 44/5981
processing : 45/5981
processing : 46/5981
processing : 47/5981
processing : 48/5981
p

processing : 379/5981
processing : 380/5981
processing : 381/5981
processing : 382/5981
processing : 383/5981
processing : 384/5981
processing : 385/5981
processing : 386/5981
processing : 387/5981
processing : 388/5981
processing : 389/5981
processing : 390/5981
processing : 391/5981
processing : 392/5981
processing : 393/5981
processing : 394/5981
processing : 395/5981
processing : 396/5981
processing : 397/5981
processing : 398/5981
processing : 399/5981
processing : 400/5981
processing : 401/5981
processing : 402/5981
processing : 403/5981
processing : 404/5981
processing : 405/5981
processing : 406/5981
processing : 407/5981
processing : 408/5981
processing : 409/5981
processing : 410/5981
processing : 411/5981
processing : 412/5981
processing : 413/5981
processing : 414/5981
processing : 415/5981
processing : 416/5981
processing : 417/5981
processing : 418/5981
processing : 419/5981
processing : 420/5981
processing : 421/5981
processing : 422/5981
processing : 423/5981
processing

processing : 752/5981
processing : 753/5981
processing : 754/5981
processing : 755/5981
processing : 756/5981
processing : 757/5981
processing : 758/5981
processing : 759/5981
processing : 760/5981
processing : 761/5981
processing : 762/5981
processing : 763/5981
processing : 764/5981
processing : 765/5981
processing : 766/5981
processing : 767/5981
processing : 768/5981
processing : 769/5981
processing : 770/5981
processing : 771/5981
processing : 772/5981
processing : 773/5981
processing : 774/5981
processing : 775/5981
processing : 776/5981
processing : 777/5981
processing : 778/5981
processing : 779/5981
processing : 780/5981
processing : 781/5981
processing : 782/5981
processing : 783/5981
processing : 784/5981
processing : 785/5981
processing : 786/5981
processing : 787/5981
processing : 788/5981
processing : 789/5981
processing : 790/5981
processing : 791/5981
processing : 792/5981
processing : 793/5981
processing : 794/5981
processing : 795/5981
processing : 796/5981
processing

processing : 1119/5981
processing : 1120/5981
processing : 1121/5981
processing : 1122/5981
processing : 1123/5981
processing : 1124/5981
processing : 1125/5981
processing : 1126/5981
processing : 1127/5981
processing : 1128/5981
processing : 1129/5981
processing : 1130/5981
processing : 1131/5981
processing : 1132/5981
processing : 1133/5981
processing : 1134/5981
processing : 1135/5981
processing : 1136/5981
processing : 1137/5981
processing : 1138/5981
processing : 1139/5981
processing : 1140/5981
processing : 1141/5981
processing : 1142/5981
processing : 1143/5981
processing : 1144/5981
processing : 1145/5981
processing : 1146/5981
processing : 1147/5981
processing : 1148/5981
processing : 1149/5981
processing : 1150/5981
processing : 1151/5981
processing : 1152/5981
processing : 1153/5981
processing : 1154/5981
processing : 1155/5981
processing : 1156/5981
processing : 1157/5981
processing : 1158/5981
processing : 1159/5981
processing : 1160/5981
processing : 1161/5981
processing 

processing : 1476/5981
processing : 1477/5981
processing : 1478/5981
processing : 1479/5981
processing : 1480/5981
processing : 1481/5981
processing : 1482/5981
processing : 1483/5981
processing : 1484/5981
processing : 1485/5981
processing : 1486/5981
processing : 1487/5981
processing : 1488/5981
processing : 1489/5981
processing : 1490/5981
processing : 1491/5981
processing : 1492/5981
processing : 1493/5981
processing : 1494/5981
processing : 1495/5981
processing : 1496/5981
processing : 1497/5981
processing : 1498/5981
processing : 1499/5981
processing : 1500/5981
processing : 1501/5981
processing : 1502/5981
processing : 1503/5981
processing : 1504/5981
processing : 1505/5981
processing : 1506/5981
processing : 1507/5981
processing : 1508/5981
processing : 1509/5981
processing : 1510/5981
processing : 1511/5981
processing : 1512/5981
processing : 1513/5981
processing : 1514/5981
processing : 1515/5981
processing : 1516/5981
processing : 1517/5981
processing : 1518/5981
processing 

...

processing : 4442/5981
processing : 4443/5981
processing : 4444/5981
processing : 4445/5981
processing : 4446/5981
processing : 4447/5981
processing : 4448/5981
processing : 4449/5981
processing : 4450/5981
processing : 4451/5981
processing : 4452/5981
processing : 4453/5981
processing : 4454/5981
processing : 4455/5981
processing : 4456/5981
processing : 4457/5981
processing : 4458/5981
processing : 4459/5981
processing : 4460/5981
processing : 4461/5981
processing : 4462/5981
processing : 4463/5981
processing : 4464/5981
processing : 4465/5981
processing : 4466/5981
processing : 4467/5981
processing : 4468/5981
processing : 4469/5981
processing : 4470/5981
processing : 4471/5981
processing : 4472/5981
processing : 4473/5981
processing : 4474/5981
processing : 4475/5981
processing : 4476/5981
processing : 4477/5981
processing : 4478/5981
processing : 4479/5981
processing : 4480/5981
processing : 4481/5981
processing : 4482/5981
processing : 4483/5981
processing : 4484/5981
processing 

processing : 4799/5981
processing : 4800/5981
processing : 4801/5981
processing : 4802/5981
processing : 4803/5981
processing : 4804/5981
processing : 4805/5981
processing : 4806/5981
processing : 4807/5981
processing : 4808/5981
processing : 4809/5981
processing : 4810/5981
processing : 4811/5981
processing : 4812/5981
processing : 4813/5981
processing : 4814/5981
processing : 4815/5981
processing : 4816/5981
processing : 4817/5981
processing : 4818/5981
processing : 4819/5981
processing : 4820/5981
processing : 4821/5981
processing : 4822/5981
processing : 4823/5981
processing : 4824/5981
processing : 4825/5981
processing : 4826/5981
processing : 4827/5981
processing : 4828/5981
processing : 4829/5981
processing : 4830/5981
processing : 4831/5981
processing : 4832/5981
processing : 4833/5981
processing : 4834/5981
processing : 4835/5981
processing : 4836/5981
processing : 4837/5981
processing : 4838/5981
processing : 4839/5981
processing : 4840/5981
processing : 4841/5981
processing 

processing : 5156/5981
processing : 5157/5981
processing : 5158/5981
processing : 5159/5981
processing : 5160/5981
processing : 5161/5981
processing : 5162/5981
processing : 5163/5981
processing : 5164/5981
processing : 5165/5981
processing : 5166/5981
processing : 5167/5981
processing : 5168/5981
processing : 5169/5981
processing : 5170/5981
processing : 5171/5981
processing : 5172/5981
processing : 5173/5981
processing : 5174/5981
processing : 5175/5981
processing : 5176/5981
processing : 5177/5981
processing : 5178/5981
processing : 5179/5981
processing : 5180/5981
processing : 5181/5981
processing : 5182/5981
processing : 5183/5981
processing : 5184/5981
processing : 5185/5981
processing : 5186/5981
processing : 5187/5981
processing : 5188/5981
processing : 5189/5981
processing : 5190/5981
processing : 5191/5981
processing : 5192/5981
processing : 5193/5981
processing : 5194/5981
processing : 5195/5981
processing : 5196/5981
processing : 5197/5981
processing : 5198/5981
processing 

processing : 5513/5981
processing : 5514/5981
processing : 5515/5981
processing : 5516/5981
processing : 5517/5981
processing : 5518/5981
processing : 5519/5981
processing : 5520/5981
processing : 5521/5981
processing : 5522/5981
processing : 5523/5981
processing : 5524/5981
processing : 5525/5981
processing : 5526/5981
processing : 5527/5981
processing : 5528/5981
processing : 5529/5981
processing : 5530/5981
processing : 5531/5981
processing : 5532/5981
processing : 5533/5981
processing : 5534/5981
processing : 5535/5981
processing : 5536/5981
processing : 5537/5981
processing : 5538/5981
processing : 5539/5981
processing : 5540/5981
processing : 5541/5981
processing : 5542/5981
processing : 5543/5981
processing : 5544/5981
processing : 5545/5981
processing : 5546/5981
processing : 5547/5981
processing : 5548/5981
processing : 5549/5981
processing : 5550/5981
processing : 5551/5981
processing : 5552/5981
processing : 5553/5981
processing : 5554/5981
processing : 5555/5981
processing 

processing : 5870/5981
processing : 5871/5981
processing : 5872/5981
processing : 5873/5981
processing : 5874/5981
processing : 5875/5981
processing : 5876/5981
processing : 5877/5981
processing : 5878/5981
processing : 5879/5981
processing : 5880/5981
processing : 5881/5981
processing : 5882/5981
processing : 5883/5981
processing : 5884/5981
processing : 5885/5981
processing : 5886/5981
processing : 5887/5981
processing : 5888/5981
processing : 5889/5981
processing : 5890/5981
processing : 5891/5981
processing : 5892/5981
processing : 5893/5981
processing : 5894/5981
processing : 5895/5981
processing : 5896/5981
processing : 5897/5981
processing : 5898/5981
processing : 5899/5981
processing : 5900/5981
processing : 5901/5981
processing : 5902/5981
processing : 5903/5981
processing : 5904/5981
processing : 5905/5981
processing : 5906/5981
processing : 5907/5981
processing : 5908/5981
processing : 5909/5981
processing : 5910/5981
processing : 5911/5981
processing : 5912/5981
processing 